### 1.0 Import Modules

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [4]:
df_rice = pd.read_csv("drive/MyDrive/COGS_Project/all_data.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (23,39,41,43,50,66,68,70,77,93,95,97,104,120,122,124,131,147,149,151,158,174,176,178,185,201,203,205,212,228,230,232,239,255,257,259,266,282,284,286,293,309,311,313,320,336,338,340,347,363,365,367,374,390,392,394,401,417,419,421,428,444,446,448,455,471,473,475,482,498,500,502,509,525,527,529,536,552,554,556,563,579,581,583,590,606,608,610,617,633,635,637,644,660,662,664,671,687,689,691,698,714,716,718,725,741,743,745,752,768,770,772,779,795,797,799,806,822,824,826,833,849,851,853,860,876,878,880,887,903,905,907,914,930,932,934,941,957,959,961,968,984,986,988,995,1011,1013,1015,1022,1038,1040,1042,1049,1065,1067,1069,1076,1092,1094,1096,1103,1119,1121,1123,1130,1146,1148,1150,1157,1173,1175,1177,1184,1200,1202,1204,1211,1227,1229,1231,1238,1254,1256,1258,1265,1281,1283,1285,1292,1308,1310,1312,1319,1335,1337,1339,1346,1362,1364,1366,1373,1389,1391,1393,1400,1416,1418,1420,1427,1443,1445,1

In [7]:
print(df_rice.shape)  # (123, 8)
df_rice.tail()

(693, 4879)


,Unnamed: 0,Open,High,Low,Close,Adj Close,Volume,Open.1,High.1,Low.1,Close.1,Adj Close.1,Volume.1,Open.2,High.2,Low.2,Close.2,Adj Close.2,Volume.2,54292099999_STATION,54292099999.0_LATITUDE,54292099999.0_LONGITUDE,54292099999.0_ELEVATION,54292099999.0_NAME,54292099999.0_TEMP,54292099999.0_TEMP_ATTRIBUTES,54292099999.0_DEWP,54292099999.0_DEWP_ATTRIBUTES,54292099999.0_SLP,54292099999.0_SLP_ATTRIBUTES,54292099999.0_STP,54292099999.0_STP_ATTRIBUTES,54292099999.0_VISIB,54292099999.0_VISIB_ATTRIBUTES,54292099999.0_WDSP,54292099999.0_WDSP_ATTRIBUTES,54292099999.0_MXSPD,54292099999.0_GUST,54292099999.0_MAX,54292099999.0_MAX_ATTRIBUTES,...,47857099999.0_VISIB_ATTRIBUTES,47857099999.0_WDSP,47857099999.0_WDSP_ATTRIBUTES,47857099999.0_MXSPD,47857099999.0_GUST,47857099999.0_MAX,47857099999.0_MAX_ATTRIBUTES,47857099999.0_MIN,47857099999.0_MIN_ATTRIBUTES,47857099999.0_PRCP,47857099999.0_PRCP_ATTRIBUTES,47857099999.0_SNDP,47857099999.0_FRSHTT,47704099999_STATION,47704099999.0_LATITUDE,47704099999.0_LONGITUDE,47704099999.0_ELEVATION,47704099999.0_NAME,47704099999.0_TEMP,47704099999.0_TEMP_ATTRIBUTES,47704099999.0_DEWP,47704099999.0_DEWP_ATTRIBUTES,47704099999.0_SLP,47704099999.0_SLP_ATTRIBUTES,47704099999.0_STP,47704099999.0_STP_ATTRIBUTES,47704099999.0_VISIB,47704099999.0_VISIB_ATTRIBUTES,47704099999.0_WDSP,47704099999.0_WDSP_ATTRIBUTES,47704099999.0_MXSPD,47704099999.0_GUST,47704099999.0_MAX,47704099999.0_MAX_ATTRIBUTES,47704099999.0_MIN,47704099999.0_MIN_ATTRIBUTES,47704099999.0_PRCP,47704099999.0_PRCP_ATTRIBUTES,47704099999.0_SNDP,47704099999.0_FRSHTT
688,2021-11-19,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,...,16.0,7.8,16.0,12.0,999.9,68.0,*,44.6,*,0.00,I,999.9,0.0,4.770410e+10,36.394611,136.406544,10.97,"KOMATSU, JA",57.5,24.0,46.2,24.0,9999.9,0.0,999.9,0.0,6.2,24.0,6.3,24.0,12.0,999.9,68.0,*,50.0,*,99.99,,999.9,10000.0
689,2021-11-20,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,...,16.0,6.8,16.0,9.9,999.9,68.0,*,51.8,*,0.00,I,999.9,0.0,4.770410e+10,36.394611,136.406544,10.97,"KOMATSU, JA",52.6,24.0,45.3,24.0,9999.9,0.0,999.9,0.0,6.2,24.0,7.5,24.0,15.0,999.9,64.4,*,42.8,*,0.00,I,999.9,0.0
690,2021-11-21,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,...,16.0,4.2,16.0,8.0,999.9,69.8,*,57.2,*,99.99,,999.9,10000.0,4.770410e+10,36.394611,136.406544,10.97,"KOMATSU, JA",57.3,24.0,48.6,24.0,9999.9,0.0,999.9,0.0,6.2,24.0,3.0,24.0,6.0,999.9,64.4,*,51.8,*,99.99,,999.9,10000.0
691,2021-11-22,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,...,14.0,15.9,14.0,21.0,999.9,68.0,*,46.4,*,99.99,,999.9,10000.0,4.770410e+10,36.394611,136.406544,10.97,"KOMATSU, JA",55.0,22.0,46.7,22.0,9999.9,0.0,999.9,0.0,5.3,22.0,12.5,22.0,20.0,28.9,66.2,*,50.0,*,99.99,,999.9,10000.0
692,2021-11-23,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,...,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.90,999.9,999.9,999.9,9.999000e+02,999.900000,999.900000,999.90,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.90,999.9,999.9,999.9


In [12]:
df_rice = df_rice.rename({"Unnamed: 0":"Date"},axis=1)

In [13]:
df_rice

,Date,Open,High,Low,Close,Adj Close,Volume,Open.1,High.1,Low.1,Close.1,Adj Close.1,Volume.1,Open.2,High.2,Low.2,Close.2,Adj Close.2,Volume.2,54292099999_STATION,54292099999.0_LATITUDE,54292099999.0_LONGITUDE,54292099999.0_ELEVATION,54292099999.0_NAME,54292099999.0_TEMP,54292099999.0_TEMP_ATTRIBUTES,54292099999.0_DEWP,54292099999.0_DEWP_ATTRIBUTES,54292099999.0_SLP,54292099999.0_SLP_ATTRIBUTES,54292099999.0_STP,54292099999.0_STP_ATTRIBUTES,54292099999.0_VISIB,54292099999.0_VISIB_ATTRIBUTES,54292099999.0_WDSP,54292099999.0_WDSP_ATTRIBUTES,54292099999.0_MXSPD,54292099999.0_GUST,54292099999.0_MAX,54292099999.0_MAX_ATTRIBUTES,...,47857099999.0_VISIB_ATTRIBUTES,47857099999.0_WDSP,47857099999.0_WDSP_ATTRIBUTES,47857099999.0_MXSPD,47857099999.0_GUST,47857099999.0_MAX,47857099999.0_MAX_ATTRIBUTES,47857099999.0_MIN,47857099999.0_MIN_ATTRIBUTES,47857099999.0_PRCP,47857099999.0_PRCP_ATTRIBUTES,47857099999.0_SNDP,47857099999.0_FRSHTT,47704099999_STATION,47704099999.0_LATITUDE,47704099999.0_LONGITUDE,47704099999.0_ELEVATION,47704099999.0_NAME,47704099999.0_TEMP,47704099999.0_TEMP_ATTRIBUTES,47704099999.0_DEWP,47704099999.0_DEWP_ATTRIBUTES,47704099999.0_SLP,47704099999.0_SLP_ATTRIBUTES,47704099999.0_STP,47704099999.0_STP_ATTRIBUTES,47704099999.0_VISIB,47704099999.0_VISIB_ATTRIBUTES,47704099999.0_WDSP,47704099999.0_WDSP_ATTRIBUTES,47704099999.0_MXSPD,47704099999.0_GUST,47704099999.0_MAX,47704099999.0_MAX_ATTRIBUTES,47704099999.0_MIN,47704099999.0_MIN_ATTRIBUTES,47704099999.0_PRCP,47704099999.0_PRCP_ATTRIBUTES,47704099999.0_SNDP,47704099999.0_FRSHTT
0,2020-01-01,999.9,999.9,999.9,999.9,999.9,999.9,999.90,999.90,999.90,999.90,999.90,999.9,999.90,999.9,999.90,999.90,999.90,999.9,5.429210e+10,42.883333,129.5,178.0,"YANJI, CH",13.4,8.0,-6.8,8.0,1029.6,8.0,995.5,8.0,18.6,8.0,8.7,8.0,12.4,28.0,20.7,,...,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.90,999.9,999.9,999.9,9.999000e+02,999.900000,999.900000,999.90,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.90,999.9,999.9,999.9
1,2020-01-02,1293.5,1293.5,1293.5,1293.5,1293.5,62.0,61.60,61.60,60.64,61.18,61.18,486873.0,559.50,567.5,558.25,560.25,560.25,49931.0,5.429210e+10,42.883333,129.5,178.0,"YANJI, CH",17.5,8.0,-3.3,8.0,1026.1,8.0,992.3,8.0,16.1,8.0,6.6,8.0,10.9,20.2,27.7,,...,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.90,999.9,999.9,999.9,9.999000e+02,999.900000,999.900000,999.90,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.90,999.9,999.9,999.9
2,2020-01-03,1289.5,1299.5,1289.5,1299.5,1299.5,1.0,61.18,64.09,61.13,63.05,63.05,885861.0,560.75,562.0,551.75,554.50,554.50,44180.0,5.429210e+10,42.883333,129.5,178.0,"YANJI, CH",20.5,8.0,2.7,8.0,1023.5,8.0,990.1,8.0,16.1,8.0,7.9,8.0,17.5,27.2,27.7,,...,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.90,999.9,999.9,999.9,9.999000e+02,999.900000,999.900000,999.90,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.90,999.9,999.9,999.9
3,2020-01-04,999.9,999.9,999.9,999.9,999.9,999.9,999.90,999.90,999.90,999.90,999.90,999.9,999.90,999.9,999.90,999.90,999.90,999.9,5.429210e+10,42.883333,129.5,178.0,"YANJI, CH",21.4,8.0,7.4,8.0,1026.8,8.0,993.4,8.0,13.8,8.0,8.7,8.0,13.6,24.3,27.7,,...,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.90,999.9,999.9,999.9,9.999000e+02,999.900000,999.900000,999.90,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.90,999.9,999.9,999.9
4,2020-01-05,999.9,999.9,999.9,999.9,999.9,999.9,999.90,999.90,999.90,999.90,999.90,999.9,999.90,999.9,999.90,999.90,999.90,999.9,5.429210e+10,42.883333,129.5,178.0,"YANJI, CH",19.3,7.0,6.6,7.0,1032.8,7.0,999.1,7.0,16.4,7.0,4.1,5.0,7.8,14.6,31.6,,...,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.90,999.9,999.9,999.9,9.999000e+02,999.900000,999.900000,999.90,999.9,999.9,

### 2.0 Var

In [ ]:
def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)
    for key,val in result[4].items():
        out[f'critical value ({key})']=val
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

In [ ]:
adf_test(df['Money'])

In [ ]:
adf_test(df['Spending'])

In [ ]:
df_difference = df.diff()
adf_test(df_difference['Money'])

In [ ]:
test_obs = 12
train = df_difference[:-test_obs]
test = df_difference[-test_obs:]

In [ ]:
for i in [1,2,3,4,5,6,7,8,9,10]:
    model = VAR(train)
    results = model.fit(i)
    print('Order =', i)
    print('AIC: ', results.aic)
    print('BIC: ', results.bic)
    print()

In [ ]:
result = model.fit(5)
result.summary()

In [ ]:
lagged_Values = train.values[-8:]


In [ ]:
pred = result.forecast(y=lagged_Values, steps=12) 

idx = pd.date_range('2015-01-01', periods=12, freq='MS')
df_forecast=pd.DataFrame(data=pred, index=idx, columns=['money_2d', 'spending_2d'])

In [ ]:
df_forecast['Money1d'] = (df['Money'].iloc[-test_obs-1]-df['Money'].iloc[-test_obs-2]) + df_forecast['money2d'].cumsum()
df_forecast['MoneyForecast'] = df['Money'].iloc[-test_obs-1] + df_forecast['Money1d'].cumsum()

In [ ]:
df_forecast['Spending1d'] = (df['Spending'].iloc[-test_obs-1]-df['Spending'].iloc[-test_obs-2]) + df_forecast['Spending2d'].cumsum()
df_forecast['SpendingForecast'] = df['Spending'].iloc[-test_obs-1] + df_forecast['Spending1d'].cumsum()

In [ ]:
test_original = df[-test_obs:]
test_original.index = pd.to_datetime(test_original.index)

In [ ]:
test_original['Money'].plot(figsize=(12,5),legend=True)
df_forecast['MoneyForecast'].plot(legend=True)

TODO:
- LSTM arch -/ 
- Stats methods arch -/
- Preprocessing Dataset + EDA
- Statistical methods apply
- LSTM apply
- LSTM + Stats apply 
- Hyperparameter tuning
- Tests